Drawing a subset of data from biendata

In [ ]:
import os, shutil, csv

In [ ]:
original_dataset_dir = 'D:\Wastesorting\Train\images_withoutrect' #directory name of your biendata
base_dir = './data_small' #create a directory for the data subset

os.mkdir(base_dir)

#creating a new folder for each set
train_dir = os.path.join(base_dir, 'train') 
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation') 
os.mkdir(validation_dir) 
test_dir = os.path.join(base_dir, 'test') 
os.mkdir(test_dir)

In [ ]:
#making a folder for each category inside train folder
#there are 204 categories with ID 1..204

for i in range(1,205):
    train_categories_dir = os.path.join(train_dir, str(i))
    os.mkdir(train_categories_dir)
    
for i in range(1,205):
    validation_categories_dir = os.path.join(validation_dir, str(i))
    os.mkdir(validation_categories_dir)

In [ ]:
#importing csv files, and change it into a list

with open('train.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile))

In [ ]:
data.pop(0)

In [ ]:
data

In [ ]:
train_copied=0 # training images copied for each category
validation_copied=0 # validation images copied for each category
last_cat=0 # records category of the last copied image
active=True # defines if we need to copy images or not in each loop
for entry in data:
    if active:
        if train_copied<10:
            shutil.copyfile(os.path.join(original_dataset_dir, entry[1]+'.png'), os.path.join(train_dir, entry[0], entry[1]+'.png'))
            train_copied+=1
        elif validation_copied<3:
            shutil.copyfile(os.path.join(original_dataset_dir, entry[1]+'.png'), os.path.join(validation_dir, entry[0], entry[1]+'.png'))
            validation_copied+=1
        else:
            active=False
        last_cat=entry[0]
    else:
        if entry[0] != last_cat:
            train_copied=0
            validation_copied=0
            shutil.copyfile(os.path.join(original_dataset_dir, entry[1]+'.png'), os.path.join(train_dir, entry[0], entry[1]+'.png'))
            train_copied+=1
            active=True

In [ ]:
#checking if it is indeed the amount
print('total training images:', len(os.listdir(train_cats_dir)))

Training model building

In [1]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras.layers import Dense, Dropout, Activation, Flatten, Lambda, BatchNormalization
from keras.layers import Convolution2D, MaxPooling2D, AveragePooling2D
from keras.engine import Input, Model
from keras.optimizers import SGD
from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
import keras.backend as K
import json
import time
import keras

Using TensorFlow backend.


In [2]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, enable=True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


As of keras 2, the module keras.layers.merge doesn't have a generic public Merge-Layer. Instead you are supposed to import the subclasses like keras.layers.Add or keras.layers.Concatenate etc. directly (or their functional interfaces with the same names lowercase: keras.layers.add, keras.layers.concatenate etc.).

In [3]:
#complete building function of Resnet x Inception v2
# we reduce # filters by factor of 8 compared to original inception-v4
nb_filters_reduction_factor = 8

def inception_resnet_v2_stem(x):
    # in original inception-resnet-v2, conv stride is 2
    x = Convolution2D(32//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(x)
    x = Convolution2D(32//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(x)
    x = Convolution2D(64//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    
    # in original inception-resnet-v2, stride is 2
    a = MaxPooling2D((3, 3), strides=(1, 1), padding='valid', data_format="channels_last")(x)
    # in original inception-resnet-v2, conv stride is 2
    b = Convolution2D(96//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(x)
   # x = merge([a, b], mode='concat', concat_axis=-1)
    x = keras.layers.concatenate([a,b],axis=-1)
    
    a = Convolution2D(64//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    a = Convolution2D(96//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(a)
    b = Convolution2D(64//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    b = Convolution2D(64//nb_filters_reduction_factor, (7, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(b)
    b = Convolution2D(64//nb_filters_reduction_factor, (1, 7), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(b)
    b = Convolution2D(96//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(b)
    x = keras.layers.concatenate([a,b],axis=-1)
    
    # in original inception-resnet-v2, conv stride should be 2
    a = Convolution2D(192//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(x)
    # in original inception-resnet-v2, stride is 2
    b = MaxPooling2D((3, 3), strides=(1, 1), padding='valid', data_format="channels_last")(x)
    x = keras.layers.concatenate([a,b],axis=-1)
    
    x = Activation('relu')(x)
    return x

def inception_resnet_v2_A(x):
    shortcut = x
    
    a = Convolution2D(32//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    
    b = Convolution2D(32//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    b = Convolution2D(32//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(b)
    
    c = Convolution2D(32//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    c = Convolution2D(48//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(c)
    c = Convolution2D(64//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(c)
    
    x = keras.layers.concatenate([a,b,c],axis=-1)
    x = Convolution2D(384//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='linear',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    
    #x = merge([shortcut, x], mode='sum')
    x = keras.layers.add([shortcut,x])
    x = Activation('relu')(x)
    
    return x

def inception_resnet_v2_reduction_A(x):
    a = MaxPooling2D((3, 3), strides=(2, 2), padding='valid', data_format="channels_last")(x)
    b = Convolution2D(384//nb_filters_reduction_factor, (3, 3), strides=(2, 2), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(x)
    c = Convolution2D(256//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    c = Convolution2D(256//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(c)
    c = Convolution2D(384//nb_filters_reduction_factor, (3, 3), strides=(2, 2), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(c)
    
    x = keras.layers.concatenate([a,b,c],axis=-1)
    
    return x
    

def inception_resnet_v2_B(x):
    shortcut = x
    
    a = Convolution2D(192//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    
    b = Convolution2D(128//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    b = Convolution2D(160//nb_filters_reduction_factor, (1, 7), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(b)
    b = Convolution2D(192//nb_filters_reduction_factor, (7, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(b)
    
    x = keras.layers.concatenate([a,b],axis=-1)
    x = Convolution2D(1154//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='linear',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    
    #x = merge([shortcut, x], mode='sum')
    x = keras.layers.add([shortcut,x])
    x = Activation('relu')(x)
    
    return x

def inception_resnet_v2_reduction_B(x):
    a = MaxPooling2D((3, 3), strides=(2, 2), padding='valid', data_format="channels_last")(x)
    b = Convolution2D(256//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    b = Convolution2D(288//nb_filters_reduction_factor, (3, 3), strides=(2, 2), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(b)
    c = Convolution2D(256//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    c = Convolution2D(288//nb_filters_reduction_factor, (3, 3), strides=(2, 2), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(c)
    d = Convolution2D(256//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    d = Convolution2D(288//nb_filters_reduction_factor, (3, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(d)
    d = Convolution2D(320//nb_filters_reduction_factor, (3, 3), strides=(2, 2), activation='relu',
                      kernel_initializer='he_normal', padding='valid', data_format="channels_last")(d)
    
    x = keras.layers.concatenate([a,b,c,d],axis=-1)
    
    return x


def inception_resnet_v2_C(x):
    shortcut = x
    
    a = Convolution2D(192//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    
    b = Convolution2D(192//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    b = Convolution2D(224//nb_filters_reduction_factor, (1, 3), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(b)
    b = Convolution2D(256//nb_filters_reduction_factor, (3, 1), strides=(1, 1), activation='relu',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(b)
    
    x = keras.layers.concatenate([a,b],axis=-1)
    x = Convolution2D(2048//nb_filters_reduction_factor, (1, 1), strides=(1, 1), activation='linear',
                      kernel_initializer='he_normal', padding='same', data_format="channels_last")(x)
    
   # x = merge([shortcut, x], mode='sum')
    x = keras.layers.add([shortcut,x])
    x = Activation('relu')(x)
    
    return x

In [4]:
img_rows, img_cols = 229, 229
img_channels = 3

nb_classes = 198

# in original inception-resnet-v2, these are 5, 10, 5, respectively
num_A_blocks = 1
num_B_blocks = 1
num_C_blocks = 1

inputs = Input(shape=(img_rows, img_cols, img_channels))

x = inception_resnet_v2_stem(inputs)
for i in range(num_A_blocks):
    x = inception_resnet_v2_A(x)
x = inception_resnet_v2_reduction_A(x)
for i in range(num_B_blocks):
    x = inception_resnet_v2_B(x)
x = inception_resnet_v2_reduction_B(x)
for i in range(num_C_blocks):
    x = inception_resnet_v2_C(x)

x = AveragePooling2D(pool_size=(4, 4), strides=(1, 1), padding='valid', data_format="channels_last")(x)
x = Dropout(0.5)(x)
x = Flatten()(x)

predictions = Dense(nb_classes, activation='softmax')(x)

model = Model(input=inputs, output=predictions)

C:\Program Files\Python36\Lib\site-packages\ipykernel_launcher.py:29: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [5]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 229, 229, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 227, 227, 4)  112         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 225, 225, 4)  148         conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 225, 225, 8)  296         conv2d_2[0][0]                   
____________________________________________________________________________________________

In [6]:
model.compile(optimizer='adam',  #optimizer=optimizers.RMSprop(lr=2e-5), could also try that
              loss='categorical_crossentropy',  
              metrics=['accuracy'])

In [7]:
#setting up all the parameter for training
batch_size = 16 #try 32, 128
epoch = 100 # try 50, 100.
data_augmentation = True

# preprocessing the data

1 Read the picture files.

2 Decode the JPEG content to RGB grids of pixels.

3 Convert these into floating-point tensors.

4 Rescale the pixel values (between 0 and 255) to the [0, 1] interval (as you know, neural networks prefer to deal with small input values).

In [8]:
import os
base_dir = './data_small'
train_dir = os.path.join(base_dir, 'train') 
validation_dir = os.path.join(base_dir, 'validation') 
test_dir = os.path.join(base_dir, 'test') 

In [9]:
#preprocessing the data implementation
#using data implementation
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
rescale=1./255,
rotation_range=40,
width_shift_range=0.2,
height_shift_range=0.2,
shear_range=0.2,
zoom_range=0.2,
horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, #train_dir is the path where you store all the validaiton folder, chnage this
    target_size = (229,229), #try 1920,1080
    batch_size = batch_size,
    class_mode = 'categorical')

validation_generator = train_datagen.flow_from_directory(
    validation_dir, #train_dir is the path where you store all the validation folder, change this
    target_size = (229,229),
    batch_size = batch_size,
    class_mode = 'categorical')



Found 1980 images belonging to 198 classes.
Found 594 images belonging to 198 classes.


In [ ]:
# saving the best model
# checkpoint

filepath="waste_sort_weights_best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [10]:
#training the model, fit the data
history = model.fit_generator(
train_generator,
steps_per_epoch=100,
epochs=epoch,
validation_data=validation_generator,
validation_steps=50,
callbacks = callbacks_list) #change here

Epoch 1/100
100/100 [==============================] - 144s 1s/step - loss: 6.0486 - accuracy: 0.0050 - val_loss: 5.2882 - val_accuracy: 0.0115
Epoch 2/100
100/100 [==============================] - 129s 1s/step - loss: 5.1491 - accuracy: 0.0094 - val_loss: 5.1829 - val_accuracy: 0.0140
Epoch 3/100
100/100 [==============================] - 128s 1s/step - loss: 5.0755 - accuracy: 0.0119 - val_loss: 5.1088 - val_accuracy: 0.0242
Epoch 4/100
100/100 [==============================] - 136s 1s/step - loss: 5.0324 - accuracy: 0.0232 - val_loss: 5.0048 - val_accuracy: 0.0285
Epoch 5/100
100/100 [==============================] - 129s 1s/step - loss: 4.9693 - accuracy: 0.0219 - val_loss: 4.8793 - val_accuracy: 0.0216
Epoch 6/100
100/100 [==============================] - 131s 1s/step - loss: 4.9715 - accuracy: 0.0269 - val_loss: 5.0189 - val_accuracy: 0.0433
Epoch 7/100
100/100 [==============================] - 135s 1s/step - loss: 4.8572 - accuracy: 0.0420 - val_loss: 4.6591 - val_accuracy:

In [11]:
model.save('wastesorting_resnet_inception_v2.h5') #model checkpoints for the last epoch

In [ ]:
#predicting the results

from keras.models import load_model
model = load_model('waste_sort_weights_best.h5')

In [ ]:
import csv
import os

#path = 'the/name/of/your/path'
path = 'D:\Wastesorting\'  
# datasets where you store the waste_sorting
original_dataset_dir = os.fsencode(path)


with open('test.csv','w') as f1:
    writer=csv.writer(f1, delimiter='\t',lineterminator='\n',)
    writer.writerow([''] + range(1000000))
    for test_img in os.listdir(original_dataset_dir):
        result = model.predict(test_img)
        writer.writerow(results)
        
#output the csv